In [12]:
import spacy
nlp = spacy.load("en_core_web_lg")

import numpy as np
import re
import importlib

import ogree_wiki as wiki
importlib.reload(wiki)

<module 'ogree_wiki' from 'c:\\Users\\thoxy\\OneDrive\\Documents\\IMT-PJENT\\Python\\ogree_wiki.py'>

In [2]:
print(wiki.conformityParameter("building","size",[20,30,40]))

salle = wiki.Room("J140",[10,10],30,[10,10,10],"+x +y","t")
print(salle.isConform())

wiki.listAllParameter("room")

True
True


['name',
 'position',
 'rotation',
 'size',
 'axisOrientation',
 'floorUnit',
 'template']

In [3]:
# Explicit lists of possible actions
ACTIONS = {
                    "ACTION_POSITIVE" : ["make","build","put","place"],
                    "ACTION_NEGATIVE" : ["remove", "delete","nuke"], 
                    "ALTER_PARAMETERS" : ["modify", "change","move"]
                    }

ACTIONS_DEFAULT = {
                    "ACTION_POSITIVE" : ["make","build","put","place"],
                    "ACTION_NEGATIVE" : ["remove", "delete"], 
                    "ALTERATION" : ["modify", "change","move"]
                    }

ACTIONS_CLI = {
                "ACTION_POSITIVE" : "+",
                "ACTION_NEGATIVE" : "-"
                }

SIMILARITY_PARAMETER = 0.5



In [4]:
def identifyAction(entry : str, dictio_actions : dict = ACTIONS_DEFAULT) -> bool:

    match = False

    # try :
    #     entry = str(input("Action word : ")).lower()

    # except ValueError :
    #     print("Invalid input, please try again")
    
    entry_doc = nlp(entry)

    if len(entry_doc) > 1 :
        print('Too many words')

    entry_doc = nlp(entry_doc[0].lemma_) # The lemmatization could and maybe should be removed here

    distance_from_action = []
    for liste in list(dictio_actions.values()) :
        distance_from_word = []
        for word in liste :
            distance_from_word.append(nlp(word)[0].similarity(entry_doc[0]))
        distance_from_action.append(max(distance_from_word))

    # make a boolean list based on the criteria match or not
    distance_from_action_criteria = list(map(lambda x : x >= SIMILARITY_PARAMETER, distance_from_action))

    if sum(distance_from_action_criteria) >= 1 :
        # if one or more actions is matching, return the "more" matching one
        action_identified = list(dictio_actions.keys())[distance_from_action.index(max(distance_from_action))]
        global operator
        operator = ACTIONS_CLI[action_identified] if action_identified != "ALTERATION" else ""
        match = True
        # print(action_identified + operator)

    # else :
    #     print("No matching action, please try again")

    return match,operator

identifyAction("make")

(True, '+')

In [5]:
def similar(word1 : str, word2 : str) -> float :
    doc1 = nlp(word1+" "+word2)
    return doc1[0].similarity(doc1[1])

In [35]:
similar("named", "called")

0.6233131885528564

In [7]:
DICTIO_CURRENT = { "site" : "IMT",
                    "building" : "A",
                    "room" : "001",
                    "rack" : "RK1"
}

HIERARCHY_WORDS = ["in"]
HIERARCHY_BASE_CLI = "/P"

def parentName(entity : str) -> str :
    ENTITIES_FULL_NAME = list(wiki.ENTITIES.keys())

    if entity in ENTITIES_FULL_NAME :
        index = ENTITIES_FULL_NAME.index(entity)
        if index == 0 :
            return entity
        else :
            return ENTITIES_FULL_NAME[index-1]
    
# TODO : refer to parent rather than current entity name
def parentAll(entity : str) -> str :
    ENTITIES_FULL_NAME = wiki.ENTITIES.keys()

    final = ""

    if entity in ENTITIES_FULL_NAME :
        current_entity = entity
        hierarchy_name_list = []
        while parentName(current_entity) != current_entity :
            current_entity = parentName(current_entity)
            hierarchy_name_list.append(DICTIO_CURRENT[current_entity])

    final += HIERARCHY_BASE_CLI +"/"
    final += "/".join(hierarchy_name_list[::-1])
    final += "/" if bool(hierarchy_name_list) == True else ""

    return final

In [8]:
parentAll("site")

'/P/'

In [36]:
def nextKeyWord(keyword_list : list, sentence : type(nlp("")), start_index : int) -> int :
    for token in sentence[start_index:] :
        if sum(list(map(lambda x : x > SIMILARITY_PARAMETER, [similar(str(token), word) for word in keyword_list]))) >= 1 : # if the token is close to a keyword
            return list(sentence).index(token)
    return len(sentence)

def lastKeyWord(keyword_list : list, sentence : type(nlp("")), start_index : int) -> int :
    for token in list(sentence)[start_index:-1:-1] :
        if sum(list(map(lambda x : x > SIMILARITY_PARAMETER, [similar(str(token), word) for word in keyword_list]))) >= 1 : # if the token is close to a keyword
            return list(sentence).index(token)
    return 0

def lastEntity(sentence : type(nlp("")), start_index : int) -> int :
    ENTITIES_FULL_NAME = list(wiki.ENTITIES.keys())
    for token in list(sentence)[start_index:-1:-1] :
        if str(token) in ENTITIES_FULL_NAME : # if the token is an entity
            return list(sentence).index(token)
    return None


In [37]:
def main() -> list :
    ENTITIES_FULL_NAME = list(wiki.ENTITIES.keys())
    FINAL_INSTRUCTIONS = []
    PROMPT_VERBS = []
    PROMPT_ENTITIES = {} 
    PROMPT_ENTITIES_RELATION = {} # the index and its relation
    PROMPT_ENTITIES_PARAMETERS = {}

    NAME_WORDS = ["called"]
    LOCATION_WORDS = ["position","at","located"]
    ROTATION_WORDS = ["rotation","turned","degree"]
    TEMPLATE_WORDS = ["template"]

    KEY_WORDS = ENTITIES_FULL_NAME + NAME_WORDS + LOCATION_WORDS + ROTATION_WORDS + TEMPLATE_WORDS

    RELATION_TYPE = ["ACTION","HIERARCHY"]
        
    print("Prompt instructions :")
    print("  - Prefer multiple easy short sentences")
    print("  - Please write only one action per entry")
    print("  - Please write explicitly the entities naming (e.g. building...)")
    print("  - Keep in mind there is no memorization for the moment")
    print("  - Please prefer writing in UPPERCASE the name given to an entity created (e.g. RACK1 )")
    print("  - Please write numbers in digits")
    natural_entry = input("Enter a prompt. Please follow the instructions given.")
    processed_entry = nlp(natural_entry)
    pos_entry = [token.pos_ for token in processed_entry]


    # identify verbs and entities
    for i in range(len(pos_entry)) :
        # if pos_entry[i] == "VERB" and identifyAction(str(processed_entry[i])) :
        #     PROMPT_VERBS.append(i)
        if str(processed_entry[i]) in ENTITIES_FULL_NAME :
            if PROMPT_ENTITIES.get(str(processed_entry[i])) != None :
                raise Exception("Two same entities were mentionned, please avoid")
            PROMPT_ENTITIES_RELATION[i] = None
            PROMPT_ENTITIES[i] = str(processed_entry[i])
            PROMPT_ENTITIES_PARAMETERS[i] = wiki.makeDictParam(str(processed_entry[i]))
            print(PROMPT_ENTITIES_PARAMETERS[i])

    if bool(PROMPT_ENTITIES.keys()) == 0 :
        raise Exception("No entity identified, please try again")
    
    # if bool(PROMPT_VERBS) == 0 :
    #     raise Exception("No action identified, please try again")
    
    # identify relations for each entity recognized
    for index in PROMPT_ENTITIES.keys() :
        checked = False
        counter = 0
        currentWordIndex = int(index)
        while checked == False and counter < 5 :
            currentWord = processed_entry[currentWordIndex].head
            currentWordIndex = list(processed_entry).index(currentWord)
            if identifyAction(str(currentWord))[0] and currentWord.pos_ == "VERB":
                PROMPT_ENTITIES_RELATION[index] = ("ACTION",currentWordIndex)
                checked = True
            elif sum(list(map(lambda x : x > SIMILARITY_PARAMETER, [similar(str(currentWord), word) for word in HIERARCHY_WORDS]))) >= 1 :
                PROMPT_ENTITIES_RELATION[index] = ("HIERARCHY",currentWordIndex)
                checked = True
            counter += 1
            
        if counter == 5 :
            raise Exception("Interpretation issue : no relation found, please try again")

        
    relation_list = [PROMPT_ENTITIES_RELATION[i][0] for i in PROMPT_ENTITIES_RELATION.keys()]
    if not "ACTION" in relation_list or relation_list.count("ACTION") > 1 : 
        raise Exception("Matching issue, please reformulate")
    index_entity_action = [index for index in PROMPT_ENTITIES_RELATION.keys() if PROMPT_ENTITIES_RELATION[index][0] == "ACTION"][0] # len should be 1
        
    for i in [index for index in PROMPT_ENTITIES_RELATION.keys() if PROMPT_ENTITIES_RELATION[index][0] == "HIERARCHY"] : # if not right hierarchy
        if ENTITIES_FULL_NAME.index(str(processed_entry[i])) > ENTITIES_FULL_NAME.index(str(processed_entry[index_entity_action])) :
            PROMPT_ENTITIES_RELATION.pop(i)
    
    # identify names
    for index in range(len(processed_entry)) : # if the name is referenced with a "called" or equivalent
        # if the token is a synonym of called
        if sum(list(map(lambda x : x > SIMILARITY_PARAMETER, [similar(str(processed_entry[index]), word) for word in NAME_WORDS]))) >= 1 :
            # if head points to entity
            if str(processed_entry[index].head) in ENTITIES_FULL_NAME and index+1 <= len(list(processed_entry)) and processed_entry[index+1].head == processed_entry[index] : 
                PROMPT_ENTITIES_PARAMETERS[list(processed_entry).index(processed_entry[index].head)]["name"] = str(processed_entry[index+1])
            
            # if head points to the action verb (associated to the entity)
            if processed_entry[index].head == processed_entry[PROMPT_ENTITIES_RELATION[index_entity_action][1]] and index+1 <= len(list(processed_entry)) and processed_entry[index+1].head == processed_entry[index] : # if head points to action
                PROMPT_ENTITIES_PARAMETERS[index_entity_action]["name"] = str(processed_entry[index+1])

            # if the name is just next to the token
            if str(processed_entry[index+1]).isupper() or pos_entry[index+1] in ["NOUN","PROPN"] :
                    index_to_assign = lastEntity(processed_entry, index-1) if not lastEntity(processed_entry, index-1) == None else index_entity_action
                    PROMPT_ENTITIES_PARAMETERS[index_to_assign]["name"] = str(processed_entry[index+1])
    
    # if all names not identified yet
    if None in [PROMPT_ENTITIES_PARAMETERS[index]["name"] for index in PROMPT_ENTITIES.keys()] :
        for index in PROMPT_ENTITIES.keys() : # if the name if right beside the entity
            if 0 <= index +1 <= len(processed_entry)-1 :
                if str(processed_entry[index+1]).isupper() or pos_entry[index+1] in ["NOUN","PROPN"] :
                    PROMPT_ENTITIES_PARAMETERS[index]["name"] = str(processed_entry[index+1])
            if 0 <= index -1 <= len(processed_entry)-1 :
                if str(processed_entry[index-1]).isupper() or pos_entry[index-1] == ["NOUN","PROPN"] :
                    PROMPT_ENTITIES_PARAMETERS[index]["name"] = str(processed_entry[index-1])

    # if nothing worked
    if None in [PROMPT_ENTITIES_PARAMETERS[index]["name"] for index in PROMPT_ENTITIES.keys()] :
        list_none = [index for index in PROMPT_ENTITIES.keys() if PROMPT_ENTITIES_PARAMETERS[index]["name"] == None]
        for index in list_none :
            PROMPT_ENTITIES_PARAMETERS[index]["name"] = DICTIO_CURRENT[PROMPT_ENTITIES[index]]
    
    # raise Exception("Missing names")
    
    # identify other parameters : at time only for building
    # TODO : make sure the parameters are refering to the right entity
    for index in range(len(processed_entry)) :

        if str(processed_entry[index]) in ENTITIES_FULL_NAME :
            continue

        if PROMPT_ENTITIES[index_entity_action] == "site" :
            continue

        # position
        if sum(list(map(lambda x : x > SIMILARITY_PARAMETER, [similar(str(processed_entry[index]), word) for word in LOCATION_WORDS]))) >= 1 and PROMPT_ENTITIES_PARAMETERS[index_entity_action]["position"] == None :
            # search for keywords to the right and left to split whole sentence into parts where to look for param entries
            next_keyword_index_position = nextKeyWord(KEY_WORDS,processed_entry,index+1)
            last_keyword_index_position = lastKeyWord(KEY_WORDS,processed_entry,index-1)
            position_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in processed_entry[index+1:next_keyword_index_position]]))
            if not len(position_list) == 2 :
                position_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in processed_entry[last_keyword_index_position:index]]))
                if not len(position_list_left) == 2 :
                    raise Exception("Wrong location format entered")
                else :
                    position_list = position_list_left
            # TODO : refine with x/y matching
            PROMPT_ENTITIES_PARAMETERS[index_entity_action]["position"] = [float(position_list[0]),float(position_list[1])]

            continue
        
        # rotation
        if sum(list(map(lambda x : x > SIMILARITY_PARAMETER, [similar(str(processed_entry[index]), word) for word in ROTATION_WORDS]))) >= 1 and PROMPT_ENTITIES_PARAMETERS[index_entity_action]["rotation"] == None :
            next_keyword_index_rotation = nextKeyWord(KEY_WORDS,processed_entry,index+1) 
            last_keyword_index_position = lastKeyWord(KEY_WORDS,processed_entry,index-1)
            next_words = processed_entry[index+1:next_keyword_index_rotation]
            last_words = processed_entry[last_keyword_index_position:index]
            
            isRotationNegative = False
            rotation_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
            isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
            if not len(rotation_list) == 1 :
                rotation_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
                isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
                if not len(position_list_left) == 1 :
                    raise Exception("Wrong rotation format entered")
                else :
                    rotation_list = rotation_list_left
            rotationFinal = -float(rotation_list[0]) if isRotationNegative else float(rotation_list[0])
            PROMPT_ENTITIES_PARAMETERS[index_entity_action]["rotation"] = rotationFinal

            continue

        # TODO : size

        # TODO : template

        # if all parameters needed are met
        if not None in [PROMPT_ENTITIES_PARAMETERS[index_entity_action][parameter_name] for parameter_name in PROMPT_ENTITIES_PARAMETERS[index_entity_action].keys()] :
            break

    # TODO : hierarchy if other entity is specified
    
    print(PROMPT_ENTITIES_PARAMETERS[index_entity_action])

    # create class instance (prefer if/else rather than match even though it's less clean)
    parameters_dict = PROMPT_ENTITIES_PARAMETERS[index_entity_action]
    final_parameter = ""

    if PROMPT_ENTITIES[index_entity_action] == "site" :
        site = wiki.Site(name = parameters_dict["name"])
        final_parameter += site.makeCLI()

    if PROMPT_ENTITIES[index_entity_action] == "building" :
        building = wiki.Building(name= parameters_dict["name"], position= parameters_dict["position"], rotation= parameters_dict["rotation"], size = parameters_dict["size"], template = parameters_dict["template"])
        boolean = building.isConform()
        if not boolean :
            raise Exception("Parameter comptability fail")
        DICTIO_CURRENT["building"] = parameters_dict["name"]
        final_parameter += building.makeCLI()

    # output formatting
    final = ""
    final += identifyAction(str(processed_entry[PROMPT_ENTITIES_RELATION[index_entity_action][1]]))[1]
    final += wiki.ENTITIES[str(processed_entry[index_entity_action])]
    final += ":"
    final += parentAll(str(processed_entry[index_entity_action]))
    final += final_parameter

    FINAL_INSTRUCTIONS.append(final)

    return FINAL_INSTRUCTIONS

In [38]:
"at 0 0 create a building named BATIMENT turned by 90 degrees"
# importlib.reload(wiki)
main()

Prompt instructions :
  - Prefer multiple easy short sentences
  - Please write only one action per entry
  - Please write explicitly the entities naming (e.g. building...)
  - Keep in mind there is no memorization for the moment
  - Please prefer writing in UPPERCASE the name given to an entity created (e.g. RACK1 )
  - Please write numbers in digits


{'name': None, 'position': None, 'rotation': None, 'size': None, 'template': None}
0


C:\Users\thoxy\AppData\Local\Temp\ipykernel_18044\1522884082.py:3: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  return doc1[0].similarity(doc1[1])


3
['0', '0']
{'name': 'BATIMENT', 'position': [0.0, 0.0], 'rotation': 90.0, 'size': None, 'template': None}


['+bd:/P/IMT/BATIMENT@[0.0, 0.0]@90.0']

In [30]:
test = nlp("at 0 0 create a building named BATIMENT turned by 90 degrees")
# test_formatted = nlp(" ".join([word.lemma_ for word in test]))
spacy.displacy.render(test, style="dep")

In [ ]:
re.findall("[-]*[0-9]+[.]*[0-9]*", "x -0. y 0")

['-0.', '0']

In [ ]:
liste = [1,2,3,4]
liste[0:3:-1]

[]